In [ ]:
import numpy as np
from typing import List, Tuple, Union
import numpy as np
from typing import List

import numpy as np
from typing import List

def gf2_matinv(A):
    """Compute the inverse of a binary matrix A over GF(2)."""
    n = A.shape[0]
    I = np.eye(n, dtype=int)
    AI = np.concatenate((A.copy(), I), axis=1)
    for i in range(n):
        if AI[i, i] == 0:
            for j in range(i + 1, n):
                if AI[j, i] == 1:
                    AI[[i, j]] = AI[[j, i]]
                    break
        for j in range(n):
            if i != j and AI[j, i] == 1:
                AI[j] ^= AI[i]
    return AI[:, n:]

def gf2_rref(A):
    """Compute the RREF of binary matrix A over GF(2)."""
    A = A.copy()
    m, n = A.shape
    i = j = 0
    M = np.eye(m, dtype=int)
    N = np.eye(n, dtype=int)
    while i < m and j < n:
        if A[i, j] == 0:
            for k in range(i+1, m):
                if A[k, j] == 1:
                    A[[i, k]] = A[[k, i]]
                    M[[i, k]] = M[[k, i]]
                    break
        if A[i, j] == 1:
            for k in range(m):
                if k != i and A[k, j] == 1:
                    A[k] ^= A[i]
                    M[k] ^= M[i]
            for k in range(n):
                if k != j and A[i, k] == 1:
                    A[:, k] ^= A[:, j]
                    N[:, k] ^= N[:, j]
            i += 1
        j += 1
    rank = i
    return A, M, N, rank


def gf2lu(A):
    """
    Perform LU decomposition of a binary matrix A over GF(2).
    Returns L, U, P such that P @ A = L @ U.
    L is lower-triangular, U is upper-triangular, and P is a permutation matrix.
    """
    m = A.shape[0]
    U = A.copy()
    L = np.eye(m, dtype=int)
    P = np.eye(m, dtype=int)

    for k in range(m - 1):
        pivot = np.where(U[k:, k] == 1)[0]
        if pivot.size == 0:
            continue
        i = pivot[0] + k
        U[[k, i], k:] = U[[i, k], k:]
        L[[k, i], :k] = L[[i, k], :k]
        P[[k, i]] = P[[i, k]]
        for j in range(k + 1, m):
            L[j, k] = U[j, k]
            if L[j, k] == 1:
                U[j, k:] ^= U[k, k:]
    return L, U, P


def symp_mat_decompose(F: np.ndarray) -> List[np.ndarray]:
    m = F.shape[0] // 2
    I = np.eye(m, dtype=int)
    O = np.zeros((m, m), dtype=int)
    def U(k): return np.vstack([np.hstack([np.eye(k, dtype=int), np.zeros((k, m-k), dtype=int)]), np.zeros((m-k, m), dtype=int)])
    def L(k): return np.vstack([np.zeros((m-k, m), dtype=int), np.hstack([np.zeros((k, m-k), dtype=int), np.eye(k, dtype=int)])])

    A, B = F[:m, :m], F[:m, m:]
    C, D = F[m:, :m], F[m:, m:]

    if (np.array_equal(A, I) and np.array_equal(C, O) and np.array_equal(D, I)) or \
       (np.array_equal(B, O) and np.array_equal(C, O)) or \
       (np.array_equal(F, np.block([[O, I], [I, O]]))):
        return [F]

    Omega = np.block([[O, I], [I, O]])
    def Elem1(Q): return np.block([[Q, np.zeros_like(Q)], [np.zeros_like(Q), gf2_matinv(Q.T)]])
    def Elem2(R): return np.block([[I, R], [O, I]])
    def Elem3(k): return np.block([[L(m-k), U(k)], [U(k), L(m-k)]])

    A = F[:m, :m]
    _, M_A, N_A, k = gf2_rref(A)
    Qleft1 = Elem1(M_A)
    Qright = Elem1(N_A)
    Fcp = Qleft1 @ F @ Qright % 2

    if k == m:
        Rright = Elem2(Fcp[:m, m:])
        Fcp = Fcp @ Rright % 2
        R = Fcp[m:, :m]
        return [gf2_matinv(Qleft1), Omega, Elem2(R), Omega, gf2_matinv(Rright), gf2_matinv(Qright)]

    Bmk = Fcp[k:m, m+k:m]
    _, M_Bmk1, _, _ = gf2_rref(Bmk)
    M_Bmk = np.block([[np.eye(k, dtype=int), np.zeros((k, m-k), dtype=int)], [np.zeros((m-k, k), dtype=int), M_Bmk1]])
    Qleft2 = Elem1(M_Bmk)
    Fcp = Qleft2 @ Fcp % 2

    E = Fcp[:k, m+k:]
    M_E = np.block([[np.eye(k, dtype=int), E], [np.zeros((m-k, k), dtype=int), np.eye(m-k, dtype=int)]])
    Qleft3 = Elem1(M_E)
    Fcp = Qleft3 @ Fcp % 2

    S = Fcp[:k, m:m+k]
    upper_block = np.hstack([S, np.zeros((k, m-k), dtype=int)])
    lower_block = np.zeros((m-k, m), dtype=int)
    Rright = Elem2(np.vstack([upper_block, lower_block]))
    Fcp = Fcp @ Rright % 2

    Fright = Omega @ Elem3(k) % 2
    Fcp = Fcp @ Fright % 2
    R = Fcp[m:, :m]

    Q = Qleft3 @ Qleft2 @ Qleft1 % 2
    return [gf2_matinv(Q), Omega, Elem2(R), Elem3(k), gf2_matinv(Rright), gf2_matinv(Qright)]




In [22]:
import numpy as np
from typing import List, Tuple, Union

def find_circuit(F: np.ndarray) -> List[Tuple[str, Union[List[int], Tuple[int, int]]]]:
    m = F.shape[0] // 2
    I = np.eye(m, dtype=int)
    O = np.zeros((m, m), dtype=int)
    Omega = np.block([[O, I], [I, O]])

    if not np.array_equal(F @ Omega @ F.T % 2, Omega):
        print("\nInvalid symplectic matrix!")
        return []

    if np.array_equal(F, np.eye(2*m, dtype=int)):
        return []

    Decomp = symp_mat_decompose(F)  # Assume this function is defined elsewhere
    circuit = []

    for mat in Decomp:
        if np.array_equal(mat, np.eye(2*m, dtype=int)):
            continue
        elif np.array_equal(mat, Omega):
            circuit.append(('H', list(range(1, m+1))))
            continue

        A = mat[:m, :m]
        B = mat[:m, m:]
        C = mat[m:, :m]
        D = mat[m:, m:]

        if np.array_equal(A, I) and np.array_equal(C, O) and np.array_equal(D, I):
            P_ind = [i+1 for i in range(m) if B[i, i] == 1]
            if P_ind:
                circuit.append(('P', P_ind))

            B_upper = np.triu((B + np.diag(np.diag(B))) % 2)
            for j in range(m):
                CZ_ind = np.where(B_upper[j] == 1)[0]
                for k in CZ_ind:
                    circuit.append(('CZ', (j+1, k+1)))

        elif np.array_equal(B, O) and np.array_equal(C, O):
            L, U, P = gf2lu(A)  # Assume gf2lu is implemented
            if not np.array_equal(P, I):
                perm = list((np.arange(m) @ P.T) + 1)
                circuit.append(('Permute', perm))

            for j in range(m):
                inds = [i for i in np.where(L[j] == 1)[0] if i != j]
                for k in inds:
                    circuit.append(('CNOT', (j+1, k+1)))

            for j in reversed(range(m)):
                inds = [i for i in np.where(U[j] == 1)[0] if i != j]
                for k in inds:
                    circuit.append(('CNOT', (j+1, k+1)))

        else:
            k = m - np.trace(A)
            Uk = np.vstack([
                np.hstack([np.eye(k, dtype=int), np.zeros((k, m-k), dtype=int)]),
                np.zeros((m-k, m), dtype=int)
            ])
            Lmk = np.vstack([
                np.zeros((k, m), dtype=int),
                np.hstack([np.zeros((m-k, k), dtype=int), np.eye(m-k, dtype=int)])
            ])
            if np.array_equal(A, Lmk) and np.array_equal(B, Uk) and \
               np.array_equal(C, Uk) and np.array_equal(D, Lmk):
                circuit.append(('H', list(range(1, k+1))))
            else:
                print("\nUnknown elementary symplectic form!")
                return []

    return circuit


In [30]:
arr = [np.array([
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
    [1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1],
    [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
]), np.array([
    [1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0],
    [0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1],
    [1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0],
    [1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1],
    [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
]), np.array([
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0],
    [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0],
    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0],
    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1]
]), np.array([
    [1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0],
    [0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1],
    [1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0],
    [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0],
    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0],
    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1]
]), np.array([
    [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1],
    [1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
    [1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0]
]), np.array([
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0],
    [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0],
    [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
    [1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0]
]), np.array([
    [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1],
    [1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
]), np.array([
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0],
    [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0],
    [0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
])]


find_circuit(arr[2])

[('Permute', [1, 6, 3, 4, 5, 2]),
 ('CNOT', (2, 1)),
 ('CNOT', (6, 1)),
 ('CNOT', (2, 6)),
 ('H', [1, 2, 3, 4, 5, 6]),
 ('P', [1, 2, 3, 4, 5]),
 ('CZ', (1, 2)),
 ('CZ', (1, 3)),
 ('CZ', (1, 4)),
 ('CZ', (1, 5)),
 ('CZ', (1, 6)),
 ('CZ', (2, 3)),
 ('CZ', (2, 4)),
 ('CZ', (2, 5)),
 ('CZ', (2, 6)),
 ('CZ', (3, 4)),
 ('CZ', (3, 5)),
 ('CZ', (4, 5)),
 ('H', [1, 2, 3, 4, 5]),
 ('CNOT', (2, 6))]